### Classification Testing Notebook
Has imports and cells for loading and psuedo-streaming SSVEP data

In [1]:
import sys
sys.path.append("..")
import time
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds, BrainFlowPresets
import numpy as np
from mvlearn.embed import CCA
import matplotlib.pyplot as plt

# Library Modules:
from modules.brainflow_stream import *
from modules.filtering import *
from modules.segmentation import *

from modules.classification import *

# from modules.preprocessing import * # Old

pygame 2.6.0 (SDL 2.28.4, Python 3.8.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
data = np.load('simulated_test_SSVEP.npy')
data.shape

(8, 15000)

In [3]:
serial_port = 'COM4' 
board_id = BoardIds.SYNTHETIC_BOARD # BoardIds.PLAYBACK_FILE_BOARD -> only if recorded with a support board (I believe...)
frequencies = [9.25, 11.25, 13.25, 15.25] # Stimulus frequencies; used for CCA & harmonic generation
buttons = ['Right', 'Left', 'Up', 'Down'] # Adds custom text to each box - must be same length as frequencies 
button_pos = [0, 2, 3, 1] # Assigns positions to custom text - must be same length as buttons
segment_duration = 4 # seconds
display = 0 # Which screen to display the stimulus paradigm on --> 0 is default

# Static Variables - Probably don't need to touch :)
harmonics = np.arange(1, 4) # Generates the 1st, 2nd, & 3rd Harmonics
sampling_rate = BoardShim.get_sampling_rate(board_id)
n_samples = sampling_rate * segment_duration 


eeg_channels = BoardShim.get_eeg_names(board_id)
channel_names = ["O1", "O2", "Oz", "Pz", "P3", "P4", "POz", "P1"]
channel_mapping = dict(zip(eeg_channels, channel_names))

# Show board information
print(f"Sampling Rate: {sampling_rate}")
print(f"Default Channels: {eeg_channels}")
print(f"Channel Mapping: {channel_mapping}")

board = BrainFlowBoardSetup(board_id, serial_port)
# board.show_params() # Logger shows this info by default - this is another method to show
board.setup()

Sampling Rate: 250
Default Channels: ['Fz', 'C3', 'Cz', 'C4', 'Pz', 'PO7', 'Oz', 'PO8', 'F5', 'F7', 'F3', 'F1', 'F2', 'F4', 'F6', 'F8']
Channel Mapping: {'Fz': 'O1', 'C3': 'O2', 'Cz': 'Oz', 'C4': 'Pz', 'Pz': 'P3', 'PO7': 'P4', 'Oz': 'POz', 'PO8': 'P1'}
Board setup and streaming started successfully


In [4]:
frequencies = [9.25, 11.25, 13.25, 15.25]
harmonics = np.arange(1, 4)
sampling_rate = 250

### Testing CCA

In [5]:
# Taking a subsection of the simulated data (1000 samples)
eeg_data = data[:, 3000:4000]


cca_classifier = SSVEPClassifier(frequencies, harmonics, sampling_rate, n_samples=len(eeg_data[1]), method='CCA', stack_harmonics=True)

# Perform CCA analysis
detected_freq, correlation = cca_classifier(eeg_data)
print(f"Detected frequency using CCA: {detected_freq} Hz with correlation: {correlation:.3f}")

Detected frequency using CCA: 11.25 Hz with correlation: 0.997


### Testing FBCCA

In [9]:
# Taking a subsection of the simulated data (1000 samples)
eeg_data = data[:, 3000:4000]

fbcca_classifier = SSVEPClassifier(frequencies, harmonics, sampling_rate, n_samples=len(eeg_data[1]), method='FBCCA', num_subbands=1)

# Perform CCA analysis
detected_freq, correlation = fbcca_classifier(eeg_data)
print(f"Detected frequency: {detected_freq} Hz with correlation: {correlation:.3f}")

Detected frequency: 11.25 Hz with correlation: 0.999


### Testing foCCA

In [10]:
# Initialize SSVEP classifier with foCCA method
focc_classifier = SSVEPClassifier(frequencies, harmonics, sampling_rate, n_samples=len(eeg_data[1]), method='foCCA')

# Perform foCCA analysis
detected_freq_focca, correlation_focca = focc_classifier(eeg_data)
print(f"Detected frequency using foCCA: {detected_freq_focca} Hz with correlation: {correlation_focca:.3f}")

Detected frequency using foCCA: None Hz with correlation: 0.000


In [22]:
board.stop()


Streaming stopped
Session released


In [7]:
segmenter = PreProcess(board, segment_duration=segment_duration)
classifier = ClassifySSVEP(frequencies, harmonics, sampling_rate, n_samples, stack_harmonics=False)
classifier_stacked = ClassifySSVEP(frequencies, harmonics, sampling_rate, n_samples, stack_harmonics=True)

while True:
    segment = segmenter.get_segment()
    if segment is not None:
        
        # print(f"Segment shape: {segment.shape}")
        eeg_segment = segment[0:8, :]

        # Step 2: Filter the data
        filtered_segment = segmenter.filter_data(eeg_segment)
        print("Filtered data shape:", filtered_segment.shape)

        # Step 3: Use CCA to match the EEG & Reference (harmonic) signals
            # Unstacked Harmonics (testing)
        detected_freq, correlation = classifier.cca_analysis(filtered_segment)
            # Stacked Harmonics (testing)
        detected_freq_stacked, correlation_stacked = classifier_stacked.cca_analysis(filtered_segment)
        
        print(f"Detected frequency: {detected_freq} Hz with correlation: {correlation}")

        # Optionally save or process the data further
        # segmenter.save_data(filtered_data, "filtered_data.csv")
        # segmenter.save_data(features, "features.csv")

    # Sleep for a while to collect new data
    time.sleep(segmenter.segment_duration)

NameError: name 'PreProcess' is not defined